TEXT

In [12]:
# IMPORTS
import random
import numpy as np
import pandas as pd

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [13]:
# Set random seed to ensure reproducibility
random.seed(37)

In [14]:
# file_path = 'instances_01_KP/low-dimensional/f1_l-d_kp_10_269'
file_path = 'instances_01_KP/low-dimensional/f2_l-d_kp_20_878'
# file_path = 'instances_01_KP/large_scale/knapPI_1_100_1000_1'

data = np.loadtxt(file_path, dtype=int, usecols=(0, 1))
col_1 = data[:, 0]
col_2 = data[:, 1]

n_items = col_1[0]
capacity = col_2[0]

values = data[1:, 0]
weights = data[1:, 1]

# print("First Column:", col_1)
# print("Second Column:", col_2)
print("number of items:", n_items)
print("max weight:", capacity)
print("values:", values)
print("weights:", weights)

number of items: 20
max weight: 878
values: [44 46 90 72 91 40 75 35  8 54 78 40 77 15 61 17 75 29 75 63]
weights: [92  4 43 83 84 68 92 82  6 44 32 18 56 83 25 96 70 48 14 58]


In [15]:
def knapsack_fitness(solution, values, weights, capacity):
    total_weight = np.dot(solution, weights)
    total_value = np.dot(solution, values)
    if total_weight > capacity:
        return 0  # Invalid solution
    return total_value

In [16]:
def generate_random_solution(length):
    return np.random.randint(2, size=length)

In [17]:
test_sol = generate_random_solution(n_items)
test_sol_fitness = knapsack_fitness(test_sol, values, weights, capacity)
print(test_sol)
print(test_sol_fitness)

[1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0]
724


In [18]:
# Store problem items in dictionary

items = {}
for i in range(n_items):
    items[i] = (values[i], weights[i])

print(items)
print(len(items))
print(items[3][1])

{0: (44, 92), 1: (46, 4), 2: (90, 43), 3: (72, 83), 4: (91, 84), 5: (40, 68), 6: (75, 92), 7: (35, 82), 8: (8, 6), 9: (54, 44), 10: (78, 32), 11: (40, 18), 12: (77, 56), 13: (15, 83), 14: (61, 25), 15: (17, 96), 16: (75, 70), 17: (29, 48), 18: (75, 14), 19: (63, 58)}
20
83


In [19]:
# Create fitness class
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# weights is positive to signify maximisation of fitness, use negative to minimise
# weights must be a tuple to allow to single and multi objective problems
# to be treated in the same way

# create individual class
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE = len(items) # number of genes in individual

def evalIndividual(individual):
    weight = 0
    value = 0
    # print(individual)
    for i in range(IND_SIZE):
        value += items[i][0] * individual[i]
        weight += items[i][1] * individual[i]
    if weight > capacity:
        return (0,)
    # print(value)
    return (value,)

toolbox = base.Toolbox() # Initialise DEAP toolbox
toolbox.register("attr_int", random.randint, 0, 1) # generate atributes of 0 or 1
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
# toolbox.register("select", tools.selBest)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evalIndividual)



def BasicEA(pop_size, n_generations):
    gen = 0
    # initialise population
    population = toolbox.population(pop_size)
    # print(population)

    ngen, cxpb, mutpb = n_generations, 0.5, 0.2  # Number of generations, crossover probability, mutation probability

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(f[0] for f in x) / len(x))
    stats.register("max", lambda x: max(f[0] for f in x))

    algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=stats, halloffame=hof, verbose=True)

    best_ind = hof[0]
    print(f"Best individual: {best_ind}, Value: {evalIndividual(best_ind)[0]}")


c:\Users\Admin\anaconda3\envs\EA\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Admin\anaconda3\envs\EA\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [20]:
BasicEA(500, 10)

gen	nevals	avg    	max
0  	500   	537.038	905
1  	290   	644.612	956
2  	277   	711.168	956
3  	287   	751.506	997
4  	297   	765.644	1018
5  	335   	780.054	1018
6  	265   	840.33 	1018
7  	309   	861.438	1018
8  	313   	882.284	1024
9  	330   	901.392	1024
10 	311   	937.474	1024
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1], Value: 1024
